# 9 yr dropout analysis redux

In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import numpy as np
import scipy.stats as sst
import os
import matplotlib.pyplot as plt

try:
    import pickle
except:
    # Python 2.7 ... harumph!
    import cPickle as pickle

from enterprise import constants as const
from enterprise.signals import parameter
from enterprise.signals import prior
from enterprise.signals import selections
from enterprise.signals import signal_base
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
from enterprise.signals import utils

from utils import models
from utils import hypermod
from utils.sample_helpers import JumpProposal, get_parameter_groups

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
from acor import acor

%matplotlib inline

## use informative priors on BWM params
 * burst epoch $t_0 \in 55421 \pm 25$ (66% CI)
 * fix sky location $(\cos\theta, \phi) = (0.10, 1.15)$
 * amplitude $\log_{10}A \in \mathcal{U}(-15, -11)$

In [2]:
anomaly_costh = 0.10345571882717139
anomaly_phi = 1.15075142923366713
anomaly_skyloc = [anomaly_costh, anomaly_phi]

anomaly_t0 = 55421.5853669
anomaly_dt0 = 25.494436791912449

# Read in data

In [3]:
ephem = 'DE421'
datadir = '/home/pbaker/nanograv/data/'
slice_yr = 9.0

In [4]:
# read in data pickles
filename = datadir + 'nano11_{}.pkl'.format(ephem)
with open(filename, "rb") as f:
    psrs = pickle.load(f)

filename = datadir + 'nano11_setpars.pkl'
with open(filename, "rb") as f:
    noise_dict = pickle.load(f)

In [5]:
psrs = models.which_psrs(psrs, slice_yr, 3)  # select pulsars

# setup models

## custom BWM w/ dropout param

In [6]:
@signal_base.function
def bwm_delay(toas, pos, log10_h=-14.0, cos_gwtheta=0.0, gwphi=0.0,
              gwpol=0.0, t0=55000, psrk=1, antenna_pattern_fn=None):
    """
    Function that calculates the earth-term gravitational-wave
    burst-with-memory signal, as described in:
    Seto et al, van haasteren and Levin, phsirkov et al, Cordes and Jenet.
    This version uses the F+/Fx polarization modes, as verified with the
    Continuous Wave and Anisotropy papers.
    :param toas: Time-of-arrival measurements [s]
    :param pos: Unit vector from Earth to pulsar
    :param log10_h: log10 of GW strain
    :param cos_gwtheta: Cosine of GW polar angle
    :param gwphi: GW azimuthal polar angle [rad]
    :param gwpol: GW polarization angle
    :param t0: Burst central time [day]
    :param antenna_pattern_fn:
        User defined function that takes `pos`, `gwtheta`, `gwphi` as
        arguments and returns (fplus, fcross)
    :return: the waveform as induced timing residuals (seconds)
    """

    # convert
    h = 10**log10_h
    gwtheta = np.arccos(cos_gwtheta)
    t0 *= const.day

    # antenna patterns
    if antenna_pattern_fn is None:
        apc = utils.create_gw_antenna_pattern(pos, gwtheta, gwphi)
    else:
        apc = antenna_pattern_fn(pos, gwtheta, gwphi)

    # grab fplus, fcross
    fp, fc = apc[0], apc[1]

    # combined polarization
    pol = np.cos(2*gwpol)*fp + np.sin(2*gwpol)*fc

    # Define the heaviside function
    heaviside = lambda x: 0.5 * (np.sign(x) + 1)

    k = np.rint(psrk)
    
    # Return the time-series for the pulsar
    return k * pol * h * heaviside(toas-t0) * (toas-t0)

## Signal blocks

In [7]:
def wn_block(vary=False):
    # define selection by observing backend
    selection = selections.Selection(selections.by_backend)

    # white noise parameters
    if vary:
        efac = parameter.Normal(1.0, 0.10)
        equad = parameter.Uniform(-8.5, -5)
        ecorr = parameter.Uniform(-8.5, -5)
    else:
        efac = parameter.Constant()
        equad = parameter.Constant()
        ecorr = parameter.Constant()

    # white noise signals
    ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
    eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
    ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

    # combine signals
    wn = ef + eq + ec

    return wn

def rn_block(prior='log-uniform', Tspan=None):
    # red noise parameters
    if prior == 'uniform':
        log10_A = parameter.LinearExp(-20, -11)
    elif prior == 'log-uniform':
        log10_A = parameter.Uniform(-20, -11)
    else:
        raise ValueError('Unknown prior for red noise amplitude!')
    gamma = parameter.Uniform(0, 7)

    # red noise signal
    powlaw = utils.powerlaw(log10_A=log10_A, gamma=gamma)
    rn = gp_signals.FourierBasisGP(powlaw, components=30, Tspan=Tspan)

    return rn

def bwm_block(t0_param, amp_prior='log-uniform',
              skyloc=None, logmin=-18, logmax=-11,
              use_k=False, name='bwm'):
    # BWM parameters
    amp_name = '{}_log10_A'.format(name)
    if amp_prior == 'uniform':
        log10_A_bwm = parameter.LinearExp(logmin, logmax)(amp_name)
    elif amp_prior == 'log-uniform':
        log10_A_bwm = parameter.Uniform(logmin, logmax)(amp_name)

    pol_name = '{}_pol'.format(name)
    pol = parameter.Uniform(0, np.pi)(pol_name)

    t0_name = '{}_t0'.format(name)
    t0 = t0_param(t0_name)

    costh_name = '{}_costheta'.format(name)
    phi_name = '{}_phi'.format(name)
    if skyloc is None:
        costh = parameter.Uniform(-1, 1)(costh_name)
        phi = parameter.Uniform(0, 2*np.pi)(phi_name)
    else:
        costh = parameter.Constant(skyloc[0])(costh_name)
        phi = parameter.Constant(skyloc[1])(phi_name)

    # BWM signal
    if use_k:
        k = parameter.Uniform(0,1) # not common, one per PSR
        bwm_wf = bwm_delay(log10_h=log10_A_bwm, t0=t0,
                                 cos_gwtheta=costh, gwphi=phi, gwpol=pol,
                                 psrk=k)
    else:
        bwm_wf = utils.bwm_delay(log10_h=log10_A_bwm, t0=t0,
                                 cos_gwtheta=costh, gwphi=phi, gwpol=pol)

    bwm = deterministic_signals.Deterministic(bwm_wf, name=name)

    return bwm

## build PTA

In [8]:
outdir = '/home/pbaker/nanograv/bwm/tests/9y_dropout'
!mkdir -p $outdir

In [9]:
amp_prior = 'log-uniform'  # for detection
t0_prior = 'anomaly'  # use Normal prior on 
bayesephem = False

# find the maximum time span to set frequency sampling
tmin = np.min([p.toas.min() for p in psrs])
tmax = np.max([p.toas.max() for p in psrs])
Tspan = tmax - tmin
print("Tspan = {:f} sec ~ {:.2f} yr".format(Tspan, Tspan/const.yr))

if t0_prior == 'uniform':
    # find clipped prior range for bwm_t0
    clip = 0.05 * Tspan
    t0min = (tmin + 2*clip)/const.day  # don't search in first 10%
    t0max = (tmax - clip)/const.day  # don't search in last 5%
    print("search for t0 in [{:.1f}, {:.1f}] MJD".format(t0min, t0max))
    t0 = parameter.Uniform(Tmin, Tmax) 
elif t0_prior == 'anomaly':
    print("search for t0 in [{:.1f} +/- {:.1f}] MJD".format(anomaly_t0, anomaly_dt0))
    t0 = parameter.Normal(anomaly_t0, anomaly_dt0)

Tspan = 283916804.981541 sec ~ 9.00 yr
search for t0 in [55421.6 +/- 25.5] MJD


In [10]:
# white noise
mod = wn_block(vary=False)

# red noise
mod += rn_block(prior=amp_prior, Tspan=Tspan)

# ephemeris model
if bayesephem:
    eph = deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)

# timing model
mod += gp_signals.TimingModel(use_svd=False)

# bwm signal
mod += bwm_block(t0,
                 skyloc = anomaly_skyloc,
                 logmin=-15,
                 amp_prior=amp_prior,
                 use_k=True)

In [11]:
pta = signal_base.PTA([mod(psr) for psr in psrs])
pta.set_default_params(noise_dict)

INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_PUPPI_efac to 1.37942
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_efac to 1.08416
INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_efac to 1.15823
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_efac to 1.12256
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_PUPPI_log10_equad to -6.53584
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_log10_equad to -6.17493
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_log10_equad to -6.5087
INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_log10_equad to -7.56768
INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_log10_ecorr to -8.15092
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_log10_ecorr to -6.09581
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_log10_ecorr to -6.29747
INFO: enterprise.signals.signal_base: Setting B1855

INFO: enterprise.signals.signal_base: Setting J1455-3330_Rcvr_800_GASP_log10_equad to -6.30898
INFO: enterprise.signals.signal_base: Setting J1455-3330_Rcvr_800_GASP_log10_ecorr to -8.23923
INFO: enterprise.signals.signal_base: Setting J1455-3330_Rcvr1_2_GASP_log10_ecorr to -5.8407
INFO: enterprise.signals.signal_base: Setting J1455-3330_Rcvr_800_GUPPI_log10_ecorr to -6.95372
INFO: enterprise.signals.signal_base: Setting J1455-3330_Rcvr1_2_GUPPI_log10_ecorr to -8.18594
INFO: enterprise.signals.signal_base: Setting J1600-3053_Rcvr1_2_GUPPI_efac to 1.06068
INFO: enterprise.signals.signal_base: Setting J1600-3053_Rcvr_800_GUPPI_efac to 1.11781
INFO: enterprise.signals.signal_base: Setting J1600-3053_Rcvr1_2_GASP_efac to 1.15386
INFO: enterprise.signals.signal_base: Setting J1600-3053_Rcvr_800_GASP_efac to 1.18528
INFO: enterprise.signals.signal_base: Setting J1600-3053_Rcvr1_2_GUPPI_log10_equad to -9.58559
INFO: enterprise.signals.signal_base: Setting J1600-3053_Rcvr1_2_GASP_log10_equad t

INFO: enterprise.signals.signal_base: Setting J1741+1351_430_ASP_efac to 0.91607
INFO: enterprise.signals.signal_base: Setting J1741+1351_L-wide_PUPPI_efac to 1.04821
INFO: enterprise.signals.signal_base: Setting J1741+1351_L-wide_ASP_efac to 1.02974
INFO: enterprise.signals.signal_base: Setting J1741+1351_430_PUPPI_efac to 1.04399
INFO: enterprise.signals.signal_base: Setting J1741+1351_430_ASP_log10_equad to -6.42403
INFO: enterprise.signals.signal_base: Setting J1741+1351_L-wide_PUPPI_log10_equad to -7.18774
INFO: enterprise.signals.signal_base: Setting J1741+1351_S-wide_ASP_log10_equad to -8.08504
INFO: enterprise.signals.signal_base: Setting J1741+1351_L-wide_ASP_log10_equad to -7.15559
INFO: enterprise.signals.signal_base: Setting J1741+1351_430_PUPPI_log10_equad to -6.58155
INFO: enterprise.signals.signal_base: Setting J1741+1351_S-wide_ASP_log10_ecorr to -7.50861
INFO: enterprise.signals.signal_base: Setting J1741+1351_430_ASP_log10_ecorr to -5.54032
INFO: enterprise.signals.si

INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GUPPI_log10_equad to -6.3098
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GASP_log10_equad to -5.69371
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GUPPI_log10_equad to -6.28187
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GASP_log10_equad to -6.51163
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GASP_log10_ecorr to -6.22503
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GASP_log10_ecorr to -6.0959
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GUPPI_log10_ecorr to -6.64199
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GUPPI_log10_ecorr to -6.35725
INFO: enterprise.signals.signal_base: Setting J2317+1439_327_ASP_efac to 1.07481
INFO: enterprise.signals.signal_base: Setting J2317+1439_430_ASP_efac to 1.24557
INFO: enterprise.signals.signal_base: Setting J2317+1439_L-wide_PUPPI_efac to 1.

In [12]:
pta.summary()

Signal Name                              Signal Class                   no. Parameters      
B1855+09_efac                            MeasurementNoise               0                   


params:
"B1855+09_L-wide_PUPPI_efac":Constant=1.37942                                             
"B1855+09_L-wide_ASP_efac":Constant=1.08416                                               
"B1855+09_430_ASP_efac":Constant=1.15823                                                  
"B1855+09_430_PUPPI_efac":Constant=1.12256                                                
__________________________________________________________________________________________
B1855+09_equad                           EquadNoise                     0                   


params:
"B1855+09_L-wide_PUPPI_log10_equad":Constant=-6.53584                                     
"B1855+09_430_PUPPI_log10_equad":Constant=-6.17493                                        
"B1855+09_L-wide_ASP_log10_equad":Constant=-6.5087              

# Sample

## sampling groups

In [13]:
# default groupings
groups = get_parameter_groups(pta)

# custiom groupings
new_groups = []

# all params
new_groups.append(list(range(len(pta.param_names))))

# per psr params
for psr in pta.pulsars:
    this_group = []
    for par in pta.param_names:
        if psr in par:
            this_group.append(pta.param_names.index(par))
    new_groups.append(this_group)

# all k params
this_group = []
for par in pta.param_names:
    if '_bwm_psrk' in par:
        this_group.append(pta.param_names.index(par))
new_groups.append(this_group)

# bwm params
this_group = []
for par in pta.param_names:
    if par.startswith('bwm_'):
        this_group.append(pta.param_names.index(par))
new_groups.append(this_group)

## initial sampler

In [14]:
# dimension of parameter space
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.1**2)

sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior,
                 cov, groups=new_groups, outDir=outdir, resume=True)

# add prior draws to proposal cycle
jp = JumpProposal(pta)
sampler.addProposalToCycle(jp.draw_from_prior, 5)
sampler.addProposalToCycle(jp.draw_from_red_prior, 10)
sampler.addProposalToCycle(jp.draw_from_bwm_prior, 10)

if bayesephem:
    sampler.addProposalToCycle(jp.draw_from_ephem_prior, 10)

## save parameter file

In [ ]:
outfile = outdir + '/params.txt'
with open(outfile, 'w') as f:
    for pname in pta.param_names:
        f.write(pname+'\n')

## Sample!

In [ ]:
N = int(3.0e+06)

sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50)

Finished 0.33 percent in 1455.151758 s Acceptance rate = 0.565375Adding DE jump with weight 50
Finished 88.03 percent in 329631.232612 s Acceptance rate = 0.475804

KeyboardInterrupt: 

ERROR: tornado.general: Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/home/pbaker/.local/opt/miniconda2/lib/python2.7/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/home/pbaker/.local/opt/miniconda2/lib/python2.7/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/pbaker/.local/opt/miniconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home/pbaker/.local/opt/miniconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/pbaker/.local/opt/miniconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/home/pbaker/.local/opt/miniconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 637, in _abor